In [14]:
import os
from pathlib import Path

import cv2
import pyvips
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [15]:
data_dir = os.path.join("..", "data", "images")
image_names = os.listdir(data_dir)

print("num images:", len(image_names))

num images: 127


In [16]:
image_paths = [os.path.join(data_dir, i) for i in image_names]

shape_collate = lambda slide: (slide.height, slide.width, slide.bands)
image_shapes = [shape_collate(pyvips.Image.new_from_file(f, access="sequential")) for f in image_paths]

In [17]:
meta_data = {
    "image_name": [name for name in image_names],
    "shape": [shape for shape in image_shapes]
    }

In [18]:
df = pd.DataFrame(meta_data)
df["height"] = df["shape"].map(lambda x: x[0])
df["width"] = df["shape"].map(lambda x: x[1])
df["channels"] = df["shape"].map(lambda x: x[2])

df = df.drop("shape", axis=1)
df.head()

,image_name,height,width,channels
0,10336.ome.tif,23663,21566,3
1,10490.ome.tif,43328,33447,3
2,10548.ome.tif,40641,40072,3
3,10565.ome.tif,46012,40061,3
4,10576.ome.tif,19540,30407,3


In [19]:
fig = px.density_contour(
    df, 
    x="width", 
    y="height",
    marginal_x="box", 
    marginal_y="box",
    width=800,
    height=500,
)
fig.show()

In [12]:
min_height, min_width = df["height"].min(), df["width"].min()

print("min height:", min_height)
print("min width:", min_width)

min height: 19116
min width: 15369


In [13]:
min_height // 50

np.int64(382)

In [9]:
def center_crop(image, crop_height, crop_width):
    height, width = image.height, image.width

    left = (width - crop_width) // 2
    top = (height - crop_height) // 2

    cropped = image.crop(left, top, crop_width, crop_height).numpy()

    return cropped

In [10]:
def downsample(image_path, crop_height, crop_width):
    new_shape = (crop_width // 50, crop_height // 50)
    
    image = pyvips.Image.new_from_file(image_path)
    image = center_crop(image, crop_height, crop_width)
    image = cv2.resize(image, new_shape)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image

In [11]:
dest_path = os.path.join("..", "assets", "tests", "cropping")

if not os.path.isdir(dest_path):
    os.makedirs(dest_path)

for path in tqdm(image_paths, desc="cropping images"):
    image_name = Path(path).stem.split(".")[0]
    downsampled = downsample(path, min_height, min_width)
    cv2.imwrite(os.path.join(dest_path, f"{image_name}.jpg"), downsampled)

cropping images:  13%|█▎        | 16/127 [02:27<17:01,  9.20s/it]


Error: unable to write to memory
  wbuffer_write: write failed
unix error: No space left on device
